In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Priors

most priors implemented in ScannerBit draw from the unit interval [0,1]. Here a random deviate $x$ from the unit interval gets transformed into arandom deviate $y$ from the target distribution $D$ via the cumulative distribution function (CDF)

$$y = F^{-1}(x)$$

## flat prior

sampling over the parameter range [a,b] by drawing a random value from the unit interval and then performing the transormation according to the CDF

$$F(y) = \int^y_a P(x) dx= \int^y_a \frac{dx}{b-a} = \frac{y-a}{b-a}$$

with the uniform distribution over [a,b] being

$$P(x) = \frac{1}{b-a}$$

This in turn leads leads to a sampled parameter

$$ y = F^{-1}(x) = (b-a)x + a$$

## double log flat join prior

The sampling distrbution looks like

$$P(x) \propto \begin{cases}
    1/|x| & \mathrm{lower} < x < \mathrm{flat_start} \\
    const & \mathrm{flat_start} \leq x \leq \mathrm{flat_end} \\
    1/x   & \mathrm{flat_end} < x < \mathrm{upper} \\
    \end{cases}
    $$

Therefore the Transformation is given by

$$F(y) = \int^y_a P(y) = \begin{cases}
    sgn(x)log(x) & \mathrm{lower} < x < \mathrm{flat_start} \\
    const \cdot
    $$

